In [ ]:
pip install sparksql-magic

In [61]:
%load_ext sparksql_magic

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [62]:
import os
from pyspark.sql import SparkSession,DataFrame
def read_data_frame_from_jdbc(spark: SparkSession, table: str) -> DataFrame:
    
    url = os.environ['DWH_URL']
    user = os.environ['DWH_USER']
    password = os.environ['DWH_PASSWORD']
    
    df = spark.read.format("jdbc") \
        .option("url", url) \
        .option("user", user) \
        .option("password", password) \
        .option("dbtable", table).load()
    return df


spark = SparkSession.builder.appName('Dashboard').config('spark.driver.extraClassPath', os.environ['RESOURCE_JAR']).getOrCreate()

read_data_frame_from_jdbc(spark,'sta_github.issues').createOrReplaceTempView("issues")
read_data_frame_from_jdbc(spark,'sta_github.pull_requests').createOrReplaceTempView("pull_requests")
read_data_frame_from_jdbc(spark,'sta_github.pull_requests_labels').createOrReplaceTempView("pull_requests_labels")
read_data_frame_from_jdbc(spark,'stg_github.pull_requests_metrics').createOrReplaceTempView("pull_requests_metrics")

 ### 1. How long do pull requests (PRs) stay open on average?

In [70]:
%%sparksql

select ROUND(AVG(pull_request_lead_time_mins)/60) pull_requests_stay_open_hours from pull_requests_metrics


pull_requests_stay_open_hours
127.0


### 2. What is the average time interval between an issue being created and the corresponding PR being merged?

In [71]:
%%sparksql

select ROUND(AVG(merge_interval)/60) pull_requests_stay_open_hours from pull_requests_metrics

pull_requests_stay_open_hours
91.0


### 3. Which labels are most commonly assigned to PRs, and what is their relative distribution?

In [65]:
%%sparksql

with label_count as (
    SELECT repo_name, label_name, count(pull_request_number) as count FROM pull_requests_labels
    Group by 1,2
)
select repo_name, label_name, count, 
round( (count/(SUM(count) over (PARTITION BY repo_name)))*100,2) as distribution_percentage 
from label_count
order by 3 desc



only showing top 20 row(s)


repo_name,label_name,count,distribution_percentage
grafana/grafana/,no-changelog,4375,16.46
grafana/grafana/,no-backport,3069,11.54
grafana/grafana/,area/frontend,2894,10.89
grafana/grafana/,area/backend,2174,8.18
grafana/grafana/,type/docs,1287,4.84
grafana/grafana/,backport,1266,4.76
grafana/grafana/,enterprise-ok,1259,4.74
grafana/grafana/,type/bug,1136,4.27
grafana/grafana/,add to changelog,908,3.42
grafana/grafana/,enterprise-failed,891,3.35


### 4. What is the average and median of the total number of lines changed per milestone?

In [69]:
%%sparksql

select milestone_number,milestone_title, 
ROUND(avg(total_lines_changed),2) avg_total_lines_changed,
median(total_lines_changed) median_total_lines_changed 
from pull_requests_metrics
Group by 1,2
order by 2 desc

only showing top 20 row(s)


milestone_number,milestone_title,avg_total_lines_changed,median_total_lines_changed
400,9.5.x,100.19,40.5
414,9.5.1,406.0,406.0
370,9.5.0,196.2,44.0
393,9.4.x,18.08,9.0
413,9.4.9,84.77,9.0
404,9.4.8,91.48,33.0
392,9.4.7,88.27,20.0
391,9.4.6,368.2,284.0
390,9.4.5,74.03,25.5
389,9.4.4,93.71,17.0
